In [1]:
import torch
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPTNeoForCausalLM, TrainingArguments, Trainer
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os
from tqdm import tqdm


In [2]:
torch.manual_seed(42)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("gpt2-medium").cuda()
# model.resize_token_embeddings(len(tokenizer))
# model.load_state_dict(torch.load('models/GPT2-med-2048-512/GPT2-med-2048-512.pt'))

# model = GPTNeoForCausalLM.from_pretrained("models/GPT-neo").cuda()

In [3]:
with open('data/test (5).wp_source', "r", encoding='utf-8-sig') as file:
    data = file.readlines()

In [4]:
import transformers
transformers.logging.set_verbosity_error()

output = []
output_w_prompt = []
for d in tqdm(data[:300]):
    sent = d
#     for i in range(12):
#         print(i)
    torch.cuda.empty_cache()

# generated = tokenizer.encode(f"<BOS> MY FATHER MEETS THE CAT  <newline>  <newline>  <newline>  One cold rainy day when my father was a little boy , he met an old  <newline>  alley cat on his street . <endprompt> <EOS>", return_tensors="pt")
    generated = tokenizer.encode(f'<BOS> {sent} <endprompt> <EOS>', return_tensors="pt").cuda()
#         print(generated)
    sample_outputs = model.generate(generated, do_sample=True, top_k=50, max_length=350, top_p=0.95,
            num_return_sequences=10, repetition_penalty=1.5, temperature=1.9, pad_token_id=tokenizer.eos_token_id)
    # sample_outputs = model.generate(generated, max_length=50)
    predicted_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    sent = predicted_text.replace('\n',' <newline> ')
    output_w_prompt.append(sent)
    output.append(sent[len(d):])

# f = open("results/med_FairyDB_test_2.txt", "w", encoding="utf-8")
f = open("results/medium_wp.txt", "w", encoding="utf-8")

for out in output:
    f.write(out + '\n')
f.close()
f = open("results/medium_wp_2.txt", "w", encoding="utf-8")
# f = open("results/med_prompt_FairyDB_test_2.txt", "w", encoding="utf-8")
for out in output_w_prompt:
    f.write(out + '\n')
f.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [38:37<00:00,  7.72s/it]
